# Introduction to NLP 01 - LAB 02
## The Dataset

In [21]:
from datasets import load_dataset
import pandas as pd

X_train, X_test, X_unsupervised = load_dataset("imdb", split=["train", "test", "unsupervised"])
print(X_train.shape, X_test.shape, X_unsupervised.shape)

Found cached dataset imdb (C:/Users/Tom/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

(25000, 2) (25000, 2) (50000, 2)


In [22]:
df_train, df_test, df_unsupervised = pd.DataFrame(X_train), pd.DataFrame(X_test), pd.DataFrame(X_unsupervised)

In [23]:
df_train['label'].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [24]:
df_test['label'].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [25]:
df_unsupervised['label'].value_counts()

-1    50000
Name: label, dtype: int64


1. How many splits does the dataset has? &
2. How big are these splits? &
3. What is the proportion of each class on the supervised splits?
    There are **3 datasets**:  
- *train* : **25000** samples in total, **12500** positive (1) and **12500** negative (0) samples.
- *test* : same on test but with different samples
- *unsupervised* : **50000** samples in total with no labels (-1)

## Naive Bayes Classifier

Implement your own naive Bayes classifier (the pseudo code can be found in the slides or the [book reference](https://web.stanford.edu/~jurafsky/slp3/)) or use [one provided by scikit-learn](https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes) combined with a [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

Go through the following steps.
1. (2 points) Take a look at the data and create an adapted preprocessing function with at least:
   1. Lower case the text.
   2. Replace punctuations with spaces (you can use `from string import punctuation` to ease your work). Think that maybe not all punctuations should be removed or replaced.

In [26]:
df_train.head(10)

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
5,I would put this at the top of my list of film...,0
6,Whoever wrote the screenplay for this movie ob...,0
7,"When I first saw a glimpse of this movie, I qu...",0
8,"Who are these ""They""- the actors? the filmmake...",0
9,This is said to be a personal film for Peter B...,0


In [27]:
from string import punctuation



def preprocessing(input : str) -> str:
    """
    Preprocesses the input string.
    """
    translator = str.maketrans(punctuation, ' ' * len(punctuation))
    input.lower().translate(translator)
    